In [ ]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import coiled

In [ ]:
cluster = coiled.Cluster(
    n_workers = 10)
client = cluster.get_client()

In [ ]:
client

In [ ]:
dask.config.set({"dataframe.convert-string": True})

## read csv data

In [ ]:
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
                     dtype = {
                     "shared_request_flag": "category",
                     "shared_match_flag": "category",
                     "access_a_ride_flag": "category",
                     "wav_request_flag": "category",
                     "wav_match_flag": "category",    
                     }, 
                     parse_dates=["request_datetime", "on_scene_datetime", # this is not working well see error below
                                  "pickup_datetime", "dropoff_datetime"]
                    )

In [ ]:
#notice the ints and floats are infered as 64
df_csv.dtypes

In [ ]:
#this is not working
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)

```python
ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

The following columns failed to properly parse as dates:

- on_scene_datetime

This is usually due to an invalid value in that column. To
diagnose and fix it's recommended to drop these columns from the
`parse_dates` keyword, and manually convert them to dates later
using `dd.to_datetime`.


```

## If we just do 
```python
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*",)
```

It will be infering date times and categories as strings, and the last category `wav_match_flag` because it has NaN is converting it to floats but the values are ["N", "Y"]

and this fails because of the dtype of `wav_match_flag`
```python
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)
```

## read parquet


In [ ]:
df_pq = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")

In [ ]:
#beauty of parquet meta - get dtypes right
df_pq.dtypes

In [ ]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_pq.tips.sum().compute() / 1e6

In [ ]:
%%time
df_pq.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

### Try 100MB partitions

In [ ]:
df_pq = df_pq.repartition(partition_size="100MB").persist()

In [ ]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

## Went from ~ 5.9 s to 137ms (~43X faster)